Step 1: Patch Extraction (Python): extract patches from all images separated into the 3 sub-types

In [1]:
import os
import glob
from tqdm.notebook import tqdm
import numpy as np
from imageio import imread, imwrite

In [19]:
indata_dir = "./indir/"
outdir = "./outdir/patches/"
# os.mkdir()

step_size=32;
patch_size=36; 

classes=['CLL','FL','MCL'];
class_struct={};

In [15]:
data_X, data_y = [], []

for i, cls in tqdm(enumerate(classes)):
    files = glob.glob(f'{indata_dir}{cls}/*.tif')
    files = sorted(files)
    fids = [f"{cls.upper()}-{fn.split('/')[-1].split('.')[0]}" for fn in files]
    
    pbar = tqdm(fids, leave=False)
    for fid in pbar:
        fp = f"{indata_dir}{cls}/{'-'.join(fid.split('-')[1:])}.tif"
        im = imread(fp)        
        nrow, ncol, ndim = im.shape
        sub_idx=0;
        data_X_patches = []
        for rr in range(0, nrow-patch_size, step_size):
            for cc in range(0, ncol-patch_size, step_size):
                for rot in [1, 2]:
                    try:
                        subim=im[rr:rr+patch_size,cc:cc+patch_size,:];
                        for j in range(i-1):
                            subim=np.rot90(subim)
#                         subfname= f'{fid}_sub_{sub_idx:04d}.tif';
#                         pbar.set_description(subfname)
                        data_X_patches.append(subim)
#                         data_y.append(i)
#                         imwrite(f'{outdir}{subfname}', subim);
                        sub_idx=sub_idx+1;
                    except Exception as e:
                        print(e)
                        continue
        data_X.append(np.array(data_X_patches))
        data_y.append(i*np.ones(sub_idx, dtype=int))

0it [00:00, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

In [18]:
np_X = np.array(data_X)
np_y = np.array(data_y)
print(f'Data shape X: {np_X.shape}\nData shape y: {np_y.shape}')

Data shape X: (374, 2752, 36, 36, 3)
Data shape y: (374, 2752)


In [20]:
np.save('./data_numpy/patch_X', np_X)
np.save('./data_numpy/patch_y', np_y)